# ML Project: Fraud Detection for STEG

The Tunisian Company of Electricity and Gas (STEG) is a public and a non-administrative company, it is responsible for delivering electricity and gas across Tunisia. The company suffered tremendous losses in the order of 200 million Tunisian Dinars due to fraudulent manipulations of meters by consumers. Using the client’s billing history, our aim is to detect and recognize clients involved in fraudulent activities.

## Data Product

- Goal: develop fraud-detection system that identifies clients involved in fraudulent activities while leaving non-fraudulent clients aside. 
- Value of Product: enhance the company’s revenues by reducing the losses caused by fraudulent activities, avoid reputation damage.
- Evaluation Metric: ROC-AUC (True Positive Rate and True Negative Rate will also be inspected).
- Baseline Model: coin toss, since no intuitive baseline model gave good results.

## Setup

In [14]:
import pandas as pd
import polars as pl
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, accuracy_score, recall_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
import pickle
from xgboost import cv

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(context='notebook', style='darkgrid')

RSEED = 12345

## Data Import and Cleaning

The following column documentation was provided by the STEG. Unfortunately, this is not a very good documentation. Some columns are left unexplained, and most explanations are not helpful.

- Client_id: Unique id for client
- District: District where the client is
- Client_catg: Category client belongs to
- Region: Area where the client is
- Creation_date: Date client joined
- Target: fraud:1 , not fraud: 0
- Invoice_date: Date of the invoice
- Tarif_type: Type of tax
- Counter_number:
- Counter_statue: takes up to 5 values such as working fine, not working, on hold statue, ect
- Counter_code:
- Reading_remarque: notes that the STEG agent takes during his visit to the client (e.g: If the counter shows something wrong, the agent gives a bad score)
- Counter_coefficient: An additional coefficient to be added when standard consumption is exceeded
- Consommation_level_1: Consumption_level_1
- Consommation_level_2: Consumption_level_2
- Consommation_level_3: Consumption_level_3
- Consommation_level_4: Consumption_level_4
- Old_index: Old index
- New_index: New index
- Months_number: Month number
- Counter_type: Type of counter

In [2]:
# Loading the data
df_client = pl.read_csv("data/client_train.csv", ignore_errors=True)
df_invoice = pl.read_csv("data/invoice_train.csv", ignore_errors=True)

# Merging
df = df_invoice.join(df_client, on = "client_id")

# Dropping duplicates
duplicates = len(df) - len(df.unique())
print(f"Number of dropped duplicate rows: {duplicates}")
df = df.unique()

# Converting counter_type from string to integer
df = df.with_columns(pl.col(["counter_type"]).replace_strict({"ELEC": 1, "GAZ": 0}))

# Cleaning counter_statue column
df = df.filter(pl.col(["counter_statue"]) <=5)

# Replace Nan with Null values
df.with_columns(pl.col(pl.Float32, pl.Float64).fill_nan(None))

# Null values
nulls = int(df.null_count().sum_horizontal().to_numpy())
print(f"Number of nulls: {nulls}")

# Ruling out transactions earlier than 2005
df = df.with_columns(pl.col(["invoice_date"]).str.to_datetime(format="%Y-%m-%d"))
df = df.with_columns(pl.col(["creation_date"]).str.to_datetime(format="%d/%m/%Y"))

# Chaning data type of target
df = df.with_columns(df["target"].cast(pl.Int8))

# Overview of the data
display(df)

Number of dropped duplicate rows: 11
Number of nulls: 0


client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,disrict,client_catg,region,creation_date,target
str,datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,datetime[μs],i8
"""train_Client_23754""",2017-06-09 00:00:00,40,6780809,0,5,6,1,122,0,0,0,3166,3288,4,0,63,11,311,1997-05-27 00:00:00,1
"""train_Client_106992""",2005-11-25 00:00:00,11,51741,0,203,6,1,314,0,0,0,9033,9347,4,1,63,11,312,1998-12-31 00:00:00,0
"""train_Client_110329""",2011-07-09 00:00:00,15,8618662,0,202,6,1,518,0,0,0,46660,47178,4,1,69,11,103,1979-02-20 00:00:00,1
"""train_Client_129249""",2008-08-22 00:00:00,11,199083,0,203,6,1,329,0,0,0,7694,8023,4,1,60,11,101,2007-06-07 00:00:00,0
"""train_Client_16844""",2008-06-27 00:00:00,11,849881,0,203,6,1,266,0,0,0,4701,4967,4,1,63,11,311,2003-08-28 00:00:00,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""train_Client_7117""",2007-05-06 00:00:00,40,6777548,0,5,6,1,0,0,0,0,341,341,2,0,62,11,301,2004-12-23 00:00:00,0
"""train_Client_42270""",2012-07-03 00:00:00,11,105049,0,207,9,1,648,0,0,0,3483,4131,4,1,62,11,301,2010-05-20 00:00:00,0
"""train_Client_64157""",2017-02-06 00:00:00,11,1301,0,532,9,1,200,100,200,2267,335290,338057,1,1,63,51,312,2008-01-21 00:00:00,0


## Statistical Properties

In [ ]:
# Statistical overview
df.describe()

### Electricity Fraud vs. Gas Fraud

There is *much* more fraudulent activity in electricity than in gas. And obviously, there in much more non-fraudulent than fraudulent activity.

In [ ]:
sns.histplot(df, x = "counter_type", hue = "target");

## Resampling

Since our dataset is highly unbalanced, we *undersample* the majority class, i.e. non-fraudulent cases.

In [3]:
# Choosing features to convert
# nominal_variables = ["tarif_type","counter_statue","reading_remarque","disrict","client_catg","region"]
#df=df.to_dummies(nominal_variables)

# Dropping features
dropped_features = ["client_id","counter_number","counter_code","invoice_date","target"]

# Defining target and features
y = np.array(df["target"])
X = np.array(df.drop(dropped_features))

# Splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=RSEED)

# Assuming X_train, y_train are your feature and target datasets
# Perform downsampling on the majority class
X_downsampled, y_downsampled = resample(X_train[y_train == 0],
                                         y_train[y_train == 0],
                                         replace=False,
                                         n_samples=np.sum(y_train == 1),
                                         random_state=RSEED)

# Concatenate the downsampled data with the minority class data
X_balanced = np.concatenate((X_downsampled, X_train[y_train == 1]))
y_balanced = np.concatenate((y_downsampled, y_train[y_train == 1]))

## Baseline Model

Our EDA gave no interesting results as to which feature could serve as a good baseline predictor. We therefore take a simple coinflip as baseline model.

In [ ]:
# Start timer
t = time.time()

# Creating column with coin flip results
number_rows = len(y_test)
y_test_pred_coin = np.random.randint(2, size=number_rows)

# Stop timer
elapsed = time.time() - t

# Plotting 
cf_matrix = confusion_matrix(y_test,y_test_pred_coin)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')
plt.title("Confusion Matrix for Coin Flip")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.show()

# Metrics
roc_auc = roc_auc_score(y_test, y_test_pred_coin)
tpr = recall_score(y_test, y_test_pred_coin)
tnr = recall_score(y_test, y_test_pred_coin, pos_label=0)

# Store metrics
coin_metrics = pl.DataFrame({
    "Model": "Coin Flip", 
    "ROC-AUC": [roc_auc], 
    "True Positive Rate": [tpr], 
    "True Negative Rate": [tnr],
    "Elapsed Time in Seconds": [elapsed],
})

# 
display(coin_metrics)

## Testing Classifiers

In [15]:
# Model list
models = {
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(n_jobs=-1),
    #"ExtraTrees": ExtraTreesClassifier(n_jobs=-1),
    #"AdaBoost": AdaBoostClassifier(),
    "LightGBM": LGBMClassifier(n_jobs=-1,verbosity=-1),
    "XGBoost": XGBClassifier(n_jobs=-1),
    "CatBoost": CatBoostClassifier(verbose=0,),
    "Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(n_jobs=-1)
}

# Initiate storage for each model
all_metrics = pl.DataFrame()

# Loop through models
for model_name, model in models.items():

    # Start timer
    t = time.time()

    # Initializing and fitting pipeline with standard scaler
    pipe = Pipeline([('scale', StandardScaler()),(model_name, model)]).fit(X_balanced, y_balanced)
    
    # Predict
    y_test_pred = pipe.predict(X_test)
    y_test_pred_prob = pipe.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    # Stop timer
    elapsed = time.time() - t
    
    # # Confusion Matrix
    # cf_matrix = confusion_matrix(y_test, y_test_pred)
    
    # # Plotting Confusion Matrix
    # sns.heatmap(cf_matrix / cf_matrix.sum(axis=1)[:, np.newaxis], annot=True, fmt='.2%', cmap='Blues')
    # plt.title(f"Confusion Matrix for {model_name}")
    # plt.xlabel("Predicted label")
    # plt.ylabel("True label")
    # plt.show()
    
    # Metrics
    roc_auc = roc_auc_score(y_test, y_test_pred_prob) if y_test_pred_prob is not None else None
    tpr = recall_score(y_test, y_test_pred)
    tnr = recall_score(y_test, y_test_pred, pos_label=0)
    
    # Store metrics
    model_metrics = pl.DataFrame({
        "Model": [model_name], 
        "ROC-AUC": [roc_auc], 
        "True Positive Rate": [tpr], 
        "True Negative Rate": [tnr],
        "Elapsed Time in Seconds": [elapsed],
    })
    
    # Concatenate
    all_metrics = pl.concat([all_metrics, model_metrics])

# Display metrics
pl.Config.set_tbl_hide_column_data_types(True)  
display(all_metrics)

Model,ROC-AUC,True Positive Rate,True Negative Rate,Elapsed Time in Seconds
"""DecisionTree""",0.807553,0.817281,0.796661,7.162922
"""RandomForest""",0.850744,0.773744,0.759178,58.744354
"""LightGBM""",0.723921,0.676871,0.644608,5.042496
"""XGBoost""",0.760436,0.692579,0.681662,3.225021
"""CatBoost""",0.777287,0.710232,0.69272,53.399082
"""Naive Bayes""",0.597626,0.994553,0.008388,1.363348
"""Logistic Regression""",0.623053,0.596075,0.585133,2.83381


## XGBoost Optimization

In [26]:
# Defining parameter grid (as dictionary)
param_grid = {"n_estimators": [1500],
            "max_depth": [10],
            'min_child_weight': [1],
            "learning_rate": [.3],
            "n_jobs": [-1],
              }

# Instantiate search and define the metric to optimize 
gs = RandomizedSearchCV(XGBClassifier(), param_grid, scoring="roc_auc", cv=2, verbose=5, n_jobs=1).fit(X_balanced, y_balanced)

# Predict
y_test_pred = gs.best_estimator_.predict(X_test)
y_test_pred_prob = gs.best_estimator_.predict_proba(X_test)[:, 1] if hasattr(gs.best_estimator_, "predict_proba") else None

# Confusion Matrix
cf_matrix = confusion_matrix(y_test, y_test_pred)

# Plotting Confusion Matrix
sns.heatmap(cf_matrix / cf_matrix.sum(axis=1)[:, np.newaxis], annot=True, fmt='.2%', cmap='Blues')
plt.title(f"Confusion Matrix for Optimized XGBoost")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.show()

# Metrics
roc_auc = roc_auc_score(y_test, y_test_pred_prob) if y_test_pred_prob is not None else None
tpr = recall_score(y_test, y_test_pred)
tnr = recall_score(y_test, y_test_pred, pos_label=0)

# Store metrics
model_metrics = pl.DataFrame({
    "Model": "Optimized XGBoost", 
    "ROC-AUC": roc_auc, 
    "True Positive Rate": tpr, 
    "True Negative Rate": tnr
})

# Present results
pl.Config.set_tbl_hide_column_data_types(True)  
display(model_metrics)

In [27]:
xgb_cv.head()

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.720464,0.000691,0.707089,0.001087
1,0.739855,0.001025,0.723134,0.000819
2,0.757512,0.001360,0.737013,0.002807
3,0.769866,0.001465,0.745563,0.000440
4,0.780871,0.001563,0.753929,0.001121


In [ ]:

# Instantiate search and define the metric to optimize 
gs = RandomizedSearchCV(XGBClassifier(), param_grid, scoring="roc_auc", cv=2, verbose=5, n_jobs=1).fit(X_balanced, y_balanced)

# Predict
y_test_pred = gs.best_estimator_.predict(X_test)
y_test_pred_prob = gs.best_estimator_.predict_proba(X_test)[:, 1] if hasattr(gs.best_estimator_, "predict_proba") else None

# Confusion Matrix
cf_matrix = confusion_matrix(y_test, y_test_pred)

# Plotting Confusion Matrix
sns.heatmap(cf_matrix / cf_matrix.sum(axis=1)[:, np.newaxis], annot=True, fmt='.2%', cmap='Blues')
plt.title(f"Confusion Matrix for Optimized XGBoost")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.show()

# Metrics
roc_auc = roc_auc_score(y_test, y_test_pred_prob) if y_test_pred_prob is not None else None
tpr = recall_score(y_test, y_test_pred)
tnr = recall_score(y_test, y_test_pred, pos_label=0)

# Store metrics
model_metrics = pl.DataFrame({
    "Model": "Optimized XGBoost", 
    "ROC-AUC": roc_auc, 
    "True Positive Rate": tpr, 
    "True Negative Rate": tnr
})

# Present results
pl.Config.set_tbl_hide_column_data_types(True)  
display(model_metrics)

In [ ]:

# Predict
y_test_pred = gs.best_estimator_.predict(X_test)
y_test_pred_prob = gs.best_estimator_.predict_proba(X_test)[:, 1] if hasattr(gs.best_estimator_, "predict_proba") else None

# Confusion Matrix
cf_matrix = confusion_matrix(y_test, y_test_pred)

# Plotting Confusion Matrix
sns.heatmap(cf_matrix / cf_matrix.sum(axis=1)[:, np.newaxis], annot=True, fmt='.2%', cmap='Blues')
plt.title(f"Confusion Matrix for Optimized XGBoost")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.show()

# Metrics
roc_auc = roc_auc_score(y_test, y_test_pred_prob) if y_test_pred_prob is not None else None
tpr = recall_score(y_test, y_test_pred)
tnr = recall_score(y_test, y_test_pred, pos_label=0)

# Store metrics
model_metrics = pl.DataFrame({
    "Model": "Optimized XGBoost", 
    "ROC-AUC": roc_auc, 
    "True Positive Rate": tpr, 
    "True Negative Rate": tnr
})

# Present results
pl.Config.set_tbl_hide_column_data_types(True)  
display(model_metrics)

## Cross-Validation and Hyperparameter Turing

In [ ]:
# Defining ranges of hyperparameter for random search
estimator_num = np.arange(1,30,2)**2
depth = [None] + list(range(1,50,5))
var_range = np.arange(1,20,3)
criterion = ["gini", "log_loss","entropy"]
learning_rate = np.linspace(0.01, 1, 6)

# Define parameter grids for each model
param_grids = {
    "DecisionTree": {
        "max_depth": depth,
        "min_samples_split": var_range,
        "min_samples_leaf": var_range,
        "criterion": criterion,
    },
    "RandomForest": {
        "n_estimators": estimator_num,
        "max_depth": depth,
        "min_samples_split": var_range,
        "min_samples_leaf": var_range,
        "n_jobs": [-1],
    },
    "ExtraTrees": {
        "n_estimators": estimator_num,
        "max_depth": depth,
        "min_samples_split": var_range,
        "min_samples_leaf": var_range,
        "criterion": criterion,
        "n_jobs": [-1],
    },
    "XGBoost": {
        "n_estimators": estimator_num,
        "max_depth": depth,
        "min_child_weight": var_range,
        "learning_rate": learning_rate,
        "n_jobs": [-1],
    }
}

# Define models
models = {
    #"DecisionTree": DecisionTreeClassifier(),
    #"RandomForest": RandomForestClassifier(),
    #"ExtraTrees": ExtraTreesClassifier(),
    "XGBoost": XGBClassifier(),
}

# Empty list to store model metrics
model_metrics_list = []
hyperparameters_list = []

# Loop over models and perform RandomizedSearchCV
for model_name, model in models.items():
    
    # Instantiate RandomizedSearchCV
    rs = GridSearchCV(model, param_grids[model_name], scoring="roc_auc", cv=5, verbose=0, n_jobs=1).fit(X_balanced, y_balanced)
    
    # Pickle model
    pickle.dump(rs.best_estimator_, open("models/"+model_name, 'wb'))

    # Predict on test data
    y_test_pred = rs.best_estimator_.predict(X_test)
    y_test_pred_prob = rs.best_estimator_.predict_proba(X_test)[:, 1] if hasattr(rs.best_estimator_, "predict_proba") else None
    
    # # Confusion Matrix
    # cf_matrix = confusion_matrix(y_test, y_test_pred)
    
    # # Plot Confusion Matrix
    # sns.heatmap(cf_matrix / cf_matrix.sum(axis=1)[:, np.newaxis], annot=True, fmt='.2%', cmap='Blues')
    # plt.title(f"Confusion Matrix for Optimized {model_name}")
    # plt.xlabel("Predicted label")
    # plt.ylabel("True label")
    # plt.show()
    
    # Metrics
    roc_auc = roc_auc_score(y_test, y_test_pred_prob) if y_test_pred_prob is not None else None
    tpr = recall_score(y_test, y_test_pred)
    tnr = recall_score(y_test, y_test_pred, pos_label=0)
    
    metrics = {
        "Model": model_name,
        "ROC-AUC": roc_auc,
        "True Positive Rate": tpr,
        "True Negative Rate": tnr,
    }

    # Append hyperparameters to model metrics
    metrics.update(rs.best_params_)

    # Store model metrics
    model_metrics_list.append(metrics)


# Convert list of metrics and hyperparameters to DataFrame
model_metrics = pl.DataFrame(model_metrics_list)

# Present results
pl.Config.set_tbl_hide_column_data_types(True)
display(model_metrics)


## Threshold Finetuning

In [ ]:
# Plot AUC/ROC curve
fpr, tpr, thresholds = roc_curve(y_test,  y_test_pred_prob)

fig, ax = plt.subplots(figsize=(6,6))
ax.plot(fpr, tpr)
ax.set_title('ROC Curve for Tuned Decision Tree')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate');

## Feature Importance

In [ ]:
# Getting feature importance plus labels
feat_importance = gs.best_estimator_.feature_importances_
x_labels = df.drop(dropped_features).columns

pl.DataFrame({"Feature": x_labels, "Importance": feat_importance}).sort(by="Importance",descending=True).head(10)

## Deep Neural Networks

In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

# Number of features (input dimension)
input_shape = [X_balanced.shape[1]]

# Define callback
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.01,
    restore_best_weights=True,
)

# Define model
model = Sequential()

# Input layer with 17 features and a hidden layer with 32 neurons
model.add(BatchNormalization(input_shape=input_shape))
model.add(Dense(1028,activation='relu')),
#model.add(BatchNormalization(input_shape=input_shape))
model.add(Dropout(rate=.3))
model.add(Dense(1028, activation='relu'))
#model.add(BatchNormalization(input_shape=input_shape)),
model.add(Dropout(rate=.3))

# Output layer with 1 neuron and sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer and binary crossentropy loss
model.compile(optimizer=Adam(learning_rate=0.001),
                loss=BinaryCrossentropy(),
                metrics=['AUC'])

# Display model summary
model.summary()

# Fit model
model.fit(X_balanced, y_balanced, epochs=100, batch_size=128, validation_split=0.2,callbacks=[early_stopping])

# Predict
#y_test_pred = model.predict(X_test)

# Evaluate
#roc_auc_score(y_test, y_test_pred)

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization_27          │ (None, 17)             │            68 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 1028)           │        18,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 1028)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 1028)           │     1,057,812 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 1028)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 1)              │         1,029 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,077,413 (4.11 MB)

 Trainable params: 1,077,379 (4.11 MB)

 Non-trainable params: 34 (136.00 B)

Epoch 1/100
3090/3090 ━━━━━━━━━━━━━━━━━━━━ 40s 13ms/step - AUC: 0.6422 - loss: 0.6305 - val_AUC: 0.0000e+00 - val_loss: 0.8899
Epoch 2/100
3090/3090 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - AUC: 0.6606 - loss: 0.6210 - val_AUC: 0.0000e+00 - val_loss: 0.9182
Epoch 3/100
3090/3090 ━━━━━━━━━━━━━━━━━━━━ 39s 12ms/step - AUC: 0.6613 - loss: 0.6204 - val_AUC: 0.0000e+00 - val_loss: 0.9517
Epoch 4/100
3090/3090 ━━━━━━━━━━━━━━━━━━━━ 39s 12ms/step - AUC: 0.6676 - loss: 0.6172 - val_AUC: 0.0000e+00 - val_loss: 0.9673
Epoch 5/100
3090/3090 ━━━━━━━━━━━━━━━━━━━━ 37s 12ms/step - AUC: 0.6675 - loss: 0.6175 - val_AUC: 0.0000e+00 - val_loss: 0.8462
Epoch 6/100
3090/3090 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - AUC: 0.6710 - loss: 0.6148 - val_AUC: 0.0000e+00 - val_loss: 0.9521
Epoch 7/100
1403/3090 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - AUC: 0.6755 - loss: 0.6135

KeyboardInterrupt: 